In [1]:
import requests
import time
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from prettytable import PrettyTable
import nltk

In [2]:
# Download VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER and Hugging Face sentiment analysis
sia = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\donel\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Fetch headlines based on topics
def fetch_headlines_by_topic(api_key, topic):
    url = f'https://newsapi.org/v2/top-headlines?country=us&category={topic}&apiKey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        return [article['title'] for article in articles]
    else:
        print(f"Error fetching news for topic '{topic}': {response.status_code}")
        return []


In [4]:
# Analyze sentiment using TextBlob
def analyze_sentiment_textblob(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    sentiment = 'Positive' if polarity > 0 else 'Negative' if polarity < 0 else 'Neutral'
    return polarity, subjectivity, sentiment

# Analyze sentiment using VADER
def analyze_sentiment_vader(text):
    scores = sia.polarity_scores(text)
    compound_score = scores['compound']
    sentiment = 'Positive' if compound_score >= 0.05 else 'Negative' if compound_score <= -0.05 else 'Neutral'
    return compound_score, sentiment

# Analyze sentiment using Hugging Face Transformers
def analyze_sentiment_huggingface(text):
    result = sentiment_pipeline(text)[0]
    return result['label'], result['score']

In [5]:
# Perform sentiment analysis for a list of headlines in a topic
def analyze_topic_sentiment(api_key, topic):
    print(f"\nFetching headlines for topic: {topic}")
    headlines = fetch_headlines_by_topic(api_key, topic)
    
    if not headlines:
        print(f"No headlines found for topic: {topic}")
        return

    # Table to display results
    table = PrettyTable()
    table.field_names = ["Headline", "TextBlob Sentiment", "VADER Sentiment", "Hugging Face Sentiment"]

    # Perform sentiment analysis for each headline
    for headline in headlines:
        tb_polarity, tb_subjectivity, tb_sentiment = analyze_sentiment_textblob(headline)
        vader_compound, vader_sentiment = analyze_sentiment_vader(headline)
        hf_label, hf_score = analyze_sentiment_huggingface(headline)

        # Add results to the table
        table.add_row([headline, tb_sentiment, vader_sentiment, hf_label])

    print(table)

In [ ]:
# Main function to handle multiple topics
def main():
    api_key = "e867900d8528496fb5c2ba8fe1a6390b"  # Replace with your NewsAPI.org API key

    # Topics to analyze (can be customized)
    topics = ["technology", "sports", "business", "entertainment", "health"]

    while True:
        for topic in topics:
            analyze_topic_sentiment(api_key, topic)

        print("\nWaiting for 5 minutes before fetching the next batch...")
        time.sleep(300)  # Wait for 5 minutes

if __name__ == "__main__":
    main()


Fetching headlines for topic: technology
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------+------------------------+
|                                                                           Headline                                                                           | TextBlob Sentiment | VADER Sentiment | Hugging Face Sentiment |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------+------------------------+
|                                           Popular video game company to close SF studio amid mass layoffs - SFGATE                                           |      Positive      |     Positive    |        NEGATIVE        |
|                                             iOS 18.2 mak